In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
beer_states_file = "Resources/beer_states.csv"
beer_states_df = pd.read_csv(csv_file)
beer_states_df.head()

,state,year,barrels,type
0,AK,2008,2067.69,On Premises
1,AK,2009,2263.65,On Premises
2,AK,2010,1929.15,On Premises
3,AK,2011,2251.02,On Premises
4,AK,2012,2312.43,On Premises


In [5]:
brewing_materials_file = "Resources/brewing_materials.csv"
brewing_materials_df = pd.read_csv(brewing_materials_file)
brewing_materials_df.head()

,data_type,material_type,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year
0,Pounds of Materials Used,Grain Products,2008,1,Malt and malt products,374165152,365300134,374165152.0,365300134.0
1,Pounds of Materials Used,Grain Products,2008,1,Corn and corn products,57563519,41647092,57563519.0,41647092.0
2,Pounds of Materials Used,Grain Products,2008,1,Rice and rice products,72402143,81050102,72402143.0,81050102.0
3,Pounds of Materials Used,Grain Products,2008,1,Barley and barley products,3800844,2362162,3800844.0,2362162.0
4,Pounds of Materials Used,Grain Products,2008,1,Wheat and wheat products,1177186,1195381,1177186.0,1195381.0


In [114]:
beer_states_df = beer_states_df[beer_states_df["state"].str.contains("total")==False]
beer_states_df = beer_states_df.rename(columns={"type":"presentation"})

presentations = beer_states_df.presentation.unique()
presentations_df = pd.DataFrame(presentations)
presentations_df = presentations_df.rename(columns={0:"presentation"})
beer_states_df

,state,year,barrels,presentation
0,AK,2008,2067.69,On Premises
1,AK,2009,2263.65,On Premises
2,AK,2010,1929.15,On Premises
3,AK,2011,2251.02,On Premises
4,AK,2012,2312.43,On Premises
5,AK,2013,2155.60,On Premises
6,AK,2014,2114.97,On Premises
7,AK,2015,4511.48,On Premises
8,AK,2016,13315.93,On Premises
9,AK,2017,15974.10,On Premises


In [50]:
brewing_materials_df = brewing_materials_df[['material_type', 'year', 'month', 'type','month_current']]
brewing_materials_df = brewing_materials_df[brewing_materials_df["material_type"].str.contains("Total")==False]
brewing_materials_df.head()

,material_type,year,month,type,month_current
0,Grain Products,2008,1,Malt and malt products,374165152
1,Grain Products,2008,1,Corn and corn products,57563519
2,Grain Products,2008,1,Rice and rice products,72402143
3,Grain Products,2008,1,Barley and barley products,3800844
4,Grain Products,2008,1,Wheat and wheat products,1177186


In [95]:
material_brewing = brewing_materials_df.material_type.unique()
material_brewing_df = pd.DataFrame(material_brewing)
material_brewing_df = material_brewing_df.rename(columns={0:"material_type"})
material_brewing_df

,material_type
0,Grain Products
1,Non-Grain Products


In [81]:
types_brewing = brewing_materials_df.type.unique()
types_brewing_df = pd.DataFrame(types_brewing)
types_brewing_df = types_brewing_df.rename(columns={0:"type"})
types_brewing_df

,type
0,Malt and malt products
1,Corn and corn products
2,Rice and rice products
3,Barley and barley products
4,Wheat and wheat products
5,Sugar and syrups
6,Hops (dry)
7,Hops (used as extracts)
8,Other


In [30]:
rds_connection_string = "postgres:nopal91@localhost:5432/beer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [67]:
engine.table_names()

['material_brewing', 'types_brewing', 'presentation']

In [96]:
material_brewing_df.to_sql(name="material_brewing", con=engine, if_exists="append", index=False)

In [84]:
types_brewing_df.to_sql(name="types_brewing", con=engine, if_exists="append", index=False)

In [85]:
presentations_df.to_sql(name="presentation", con=engine, if_exists="append", index=False)

In [104]:
#get the DataFrame from sql with the id created from the data base.
types_brewing_df = pd.read_sql_query('select * from types_brewing', con=engine)
material_brewing_df = pd.read_sql_query('select * from material_brewing', con=engine)
material_brewing_df

,id_material,material_type
0,1,Grain Products
1,2,Non-Grain Products


In [103]:
# merge the data frames

merged_brewing_materials= pd.merge(brewing_materials_df, types_brewing_df, how='left', on='type')
merged_brewing_materials= pd.merge(merged_brewing_materials, material_brewing_df, how='left', on='material_type')
merged_brewing_materials

,material_type,year,month,type,month_current,id_types,id_material
0,Grain Products,2008,1,Malt and malt products,374165152,1,1
1,Grain Products,2008,1,Corn and corn products,57563519,2,1
2,Grain Products,2008,1,Rice and rice products,72402143,3,1
3,Grain Products,2008,1,Barley and barley products,3800844,4,1
4,Grain Products,2008,1,Wheat and wheat products,1177186,5,1
5,Non-Grain Products,2008,1,Sugar and syrups,78358212,6,2
6,Non-Grain Products,2008,1,Hops (dry),4506546,7,2
7,Non-Grain Products,2008,1,Hops (used as extracts),621912,8,2
8,Non-Grain Products,2008,1,Other,1291615,9,2
9,Grain Products,2008,2,Malt and malt products,355687578,1,1


In [107]:
presentations_df = pd.read_sql_query('select * from presentation', con=engine)
presentations_df

,id_presentation,presentation
0,1,On Premises
1,2,Bottles and Cans
2,3,Kegs and Barrels
3,4,On Premises
4,5,Bottles and Cans
5,6,Kegs and Barrels


In [116]:
merged_beer_states = pd.merge(beer_states_df, presentations_df, how='left', on='presentation')
merged_beer_states

,state,year,barrels,presentation
0,AK,2008,2067.69,On Premises
1,AK,2009,2263.65,On Premises
2,AK,2010,1929.15,On Premises
3,AK,2011,2251.02,On Premises
4,AK,2012,2312.43,On Premises
5,AK,2013,2155.60,On Premises
6,AK,2014,2114.97,On Premises
7,AK,2015,4511.48,On Premises
8,AK,2016,13315.93,On Premises
9,AK,2017,15974.10,On Premises
